ВАРИАНТ 3.4

Необходимые функции:

In [2]:
import numpy as np

#возвращает максимально возможное число корней по теореме Декарта
def Descartes(a:list):
    coefs = [i for i in a if i != 0]
    sign_changes = 0
    for i, c in enumerate(a[1:], start=1):
        if np.sign(c) != np.sign(a[i-1]):
            sign_changes += 1
    return sign_changes        


#возвращает число корней на отрезке [left, right]
def Sturm(a:np.ndarray, left, right):
    sequence = []    #ряд Штурма
    sequence.append(a)
    sequence.append(np.polyder(a))
    for i in range(2, len(a)):
        sequence.append(np.polymul(np.polydiv(sequence[i-2], sequence[i-1])[1], -1))
    
    #значения ряда Штурма на концах отрезка 
    seq_left = [np.polyval(i, left) for i in sequence]
    seq_right = [np.polyval(i, right) for i in sequence]
    
    seq_left = [i for i in seq_left if i != 0]
    seq_right = [i for i in seq_right if i != 0]

    sign_changes_left = 0
    for i, c in enumerate(seq_left[1:], start=1):
        if np.sign(c) != np.sign(seq_left[i-1]):
            sign_changes_left += 1
            
    sign_changes_right = 0
    for i, c in enumerate(seq_right[1:], start=1):
        if np.sign(c) != np.sign(seq_right[i-1]):
            sign_changes_right += 1
            
    return sign_changes_left - sign_changes_right


#возвращает список отрезков локализации
def bisection(a:np.ndarray, left, right):
    n = Sturm(a, left, right)
    if n == 0:
        return []
    if n == 1:
        return [(left, right)]
    if n >= 2:
        mid = (left + right)/2
        return bisection(a, left, mid) + bisection(a, mid, right)
    

Входные данные:

In [3]:
gamma0 = 3
r0 = 100
u0 = -4.677611e5
p0 = 0
c0 = 1.972e5

gamma3 = 3
r3 = 21.80089
u3 = 0
p3 = 5.176613e12
c3 = 1.972e5

#вспомогательные параметры h, alpha:
h0 = (gamma0 + 1)/(gamma0 - 1)
h3 = (gamma3 + 1)/(gamma3 - 1)
alpha0 = r0*c0**2*(h0 - 1)
alpha3 = r3*c3**2*(h3 - 1)

Подсчет коэффициентов алгебраического уравнения:

In [4]:
#промежуточные параметры v, x, beta0, beta3, delta0, delta3, e:
v = u0 - u3
x = p3/(r0*r3*v**2)
beta0 = alpha0/(r0*r3*v**2)
beta3 = alpha3/(r0*r3*v**2)
delta0 = (h0 - 1)*r3
delta3 = (h3 - 1)*r0
e = p0/(r0*r3*v**2)

a = [0] * 7
a[0] = (delta0*h3 - delta3*h0)**2
a[1] = 2*(h3*delta0**2*(beta3 - h3*e) + h0*delta3**2*(beta0 - x*h0) - h0*h3*(delta0*h3 + delta3*h0) - delta0*delta3*(h0*beta3 + h3*beta0) + delta0*delta3*h0*h3*(x + e))
a[2] = h0**2*h3**2 + delta0**2*(h3**2*e**2 + beta3**2 - 4*beta3*h3*e) + delta3**2*(beta0**2 + h0**2*x**2 - 4*beta0*x*h0) - 2*delta0*h3*(2*beta3*h0 + h3*beta0 - e*h0*h3) - 2*delta3*h0*(2*beta0*h3 + h0*beta3 - h0*h3*x) - 2*delta0*delta3*(x*e*h0*h3 + beta0*beta3) + 2*delta0*delta3*(x + e)*(beta3*h0 + h3*beta0)
a[3] = 2*(h0*h3*(beta0*h3 + beta3*h0) + delta0**2*e*beta3*(h3*e - beta3) + (x*h0 - beta0)*delta3**2*beta0*x - delta0*(h0*beta3**2 - e*beta0*h3**2) - 2*delta0*h3*beta3*(beta0 - e*h0) - delta3*(h3*beta0**2 - x*beta3*h0**2) - 2*delta3*beta0*h0*(beta3 - x*h3) - e*delta0*delta3*x*(h0*beta3 + h3*beta0) + delta0*delta3*beta0*beta3*(x + e))
a[4] = beta0**2*h3**2 + beta3**2*h0**2 + 4*beta0*beta3*h0*h3 + delta0**2*beta3**2*e**2 + delta3**2*beta0**2*x**2 - 2*delta0*(beta0*beta3**2 - e*beta3**2*h0 - 2*e*h3*beta0*beta3) - 2*delta3*beta0*(beta0*(beta3 - x*h3) - 2*beta3*x*h0) - 2*e*delta0*delta3*beta0*beta3*x
a[5] = 2*(beta0*beta3*(beta0*h3 + beta3*h0) + beta0*beta3*(delta0*e*beta3 + delta3*x*beta0))
a[6] = beta0**2*beta3**2

#дальше будем работать с numpy.ndarray вместо списка list
a = np.array(a)

Локализация корней:

1. Кольцо корней:

In [5]:
#кольцо
A = max(abs(i) for i in a[1:6])
B = max(abs(i) for i in a[0:5])
left = abs(a[6])/(abs(a[6]) + B)
right = 1 + A/abs(a[0])
print(left, right)

8.583223249470204e-15 1.097498026679439


2. Теорема Декарта:

In [6]:
print(Descartes(a))

2


3. Теорема Штурма:

In [7]:
print(Sturm(a, left, right))

2


4. Локализация отрезков методом половинного деления:

In [8]:
sections = bisection(a, left, right)
print(sections)

[(8.583223249470204e-15, 0.06859362666747298), (0.06859362666747298, 0.1371872533349374)]


Поиск корня с заданной точностью:

1. Метод простой итерации:
    Для сходимости МПИ при любом начальном приближении требуется строгая ограниченность производной. Будем добиваться выполнения этого условия следующим образом: уменьшаем отрезок локализации до тех пор, пока вторая производная не будет обращаться в ноль на всей области. Тогда первая производная монотонна и имеет максимум в одном из двух концов отрезка локализации.

In [9]:
#построение итерационного процесса и проверка выполнения условий сходимости
n = len(a) - 1
fi = np.array(a) #copy
fi[n-1] += 1    #коэффициенты функции fi(y) в уравнении y = fi(y)
fi_d1 = np.polyder(fi)    #первая производная
fi_d2 = np.polyder(fi_d1)    #вторая производная
left, right = sections[0]    #выберем отрезок локализации первого корня

#локализуем отрезок для выполнения условий сходимости
while True and Sturm(fi_d2, left, right):    #сужаем отрезок, пока первая производная не станет монотонной
    mid = (left + right)/2
    left_val = np.polyval(a, left)
    mid_val = np.polyval(a, mid)
    right_val = np.polyval(a, right)
    if np.sign(left_val*mid_val) == -1:
        right = mid
    else:
        left = mid
#проверим, что max(|fi_d1|) = q < 1
q = max(abs(np.polyval(fi_d1, left)), abs(np.polyval(fi_d1, right)))
print("Производная функции метода простой итерации ограничена числом", q, ", то есть условие сходимости выполнено.")    #первая производная действительно ограничена числом q < 1

#ищем корень до заданной точности
print("Вычислим значение корня y1 с разной точностью:")
precision1 = 1e-4
precision2 = 1e-5
precision3 = 1e-6

y0 = (left + right)/2    #произвольное начальное приближение
y_prev = y0
while True:
    y = np.polyval(fi, y_prev)
    if abs(y - y_prev)/(1 - q) <= precision1 :
        break
    y_prev = y
print("С точностью 1e-04", ":", y)

y_prev = y0 = y
while True:
    y = np.polyval(fi, y_prev)
    if abs(y - y_prev)/(1 - q) <= precision2 :
        break
    y_prev = y
print("С точностью", precision2, ":", y)

y_prev = y0 = y
while True:
    y = np.polyval(fi, y_prev)
    if abs(y - y_prev)/(1 - q) <= precision3 :
        break
    y_prev = y
print("С точностью", precision3, ":", y)
Y1 = y

Производная функции метода простой итерации ограничена числом 0.9998085062532166 , то есть условие сходимости выполнено.
Вычислим значение корня y1 с разной точностью:
С точностью 1e-04 : 0.018364866044581863
С точностью 1e-05 : 0.018309237738945893
С точностью 1e-06 : 0.01830361173023717


2. Метод Ньютона:

    С помощью итерационного метода Ньютона найдем значение второго корня - y2. 

In [10]:
n = len(a) - 1
f = np.array(a)    #copy #коэффициенты функции f(y) в уравнении f(y) = 0
f_d1 = np.polyder(f)    #первая производная
f_d2 = np.polyder(f_d1)    #вторая производная
left, right = sections[1]    #отрезок локализации корня y2
while True and Sturm(f_d1, left, right) and Sturm(f_d2, left, right):    #сужаем отрезок локализации, пока первая и вторая производные не станут знакопостоянными
    mid = (left + right)/2
    left_val = np.polyval(f, left)
    mid_val = np.polyval(f, mid)
    right_val = np.polyval(f, right)
    if np.sign(left_val*mid_val) == -1:
        right = mid
    else:
        left = mid

#проверим условие сходимости
print("Выполнение условия для начального приближения f(y0)*f''(y0) > 0 -", np.sign(np.polyval(f, right)*np.polyval(f_d2, right)) == 1)    #выбираем начальное приближение из условия f(y0)*f''(y0) > 0

#ищем корень до заданной точности

y0 = right
y_prev = y0
while True:
    y = y_prev - np.polyval(f, y_prev)/np.polyval(f_d1, y_prev)
    if abs(y-y_prev) < precision1:
        break
    y_prev = y
print("С точностью 1e-04", ":", y)

y_prev = y0 = y
while True:
    y = y_prev - np.polyval(f, y_prev)/np.polyval(f_d1, y_prev)
    if abs(y-y_prev) < precision2:
        break
    y_prev = y
print("С точностью", precision2, ":", y) 

y_prev = y0 = y
while True:
    y = y_prev - np.polyval(f, y_prev)/np.polyval(f_d1, y_prev)
    if abs(y-y_prev) < precision1:
        break
    y_prev = y
print("С точностью", precision3, ":", y)
Y2 = y

Выполнение условия для начального приближения f(y0)*f''(y0) > 0 - True
С точностью 1e-04 : 0.08846626972065016
С точностью 1e-05 : 0.08846626770589816
С точностью 1e-06 : 0.08846626770589794


5. Решение исходной задачи - определим скорости ударных волн D0 и D3:


In [11]:
print("Y1 =", Y1)

Y1 = 0.01830361173023717


In [12]:
p1 = p2 = Y1*r0*r3*v**2
r1 = r0*(p1*h0 + alpha0)/(p1 + alpha0)
r2 = r3*(p2*h3 + alpha3)/(p2 + alpha3)
u1 = u0 + np.sqrt((p1 - p0)*(r1 - r0)/(r1*r0))
u2 = u3 - np.sqrt((p2 - p3)*(r2 - r3)/(r2*r3))
D0 = (r0*u0 - r1*u1)/(r0 - r1)
D3 = (r2*u2 - r3*u3)/(r2 - r3)
print("D0 =", D0, "см/с;")
print("D3 =", D3, "см/с.")

D0 = -5694.087733316119 см/с;
D3 = -584721.6615972444 см/с.


In [13]:
print("Y2 =", Y2)

Y2 = 0.08846626770589794


In [14]:
p1 = p2 = Y2*r0*r3*v**2
r1 = r0*(p1*h0 + alpha0)/(p1 + alpha0)
r2 = r3*(p2*h3 + alpha3)/(p2 + alpha3)
u1 = u0 + np.sqrt((p1 - p0)*(r1 - r0)/(r1*r0))
u2 = u3 - np.sqrt((p2 - p3)*(r2 - r3)/(r2*r3))
D0 = (r0*u0 - r1*u1)/(r0 - r1)
D3 = (r2*u2 - r3*u3)/(r2 - r3)
print("D0 =", D0, "см/с;")
print("D3 =", D3, "см/с.")

D0 = 471847.27697507635 см/с;
D3 = -1852163.5812543835 см/с.
